In [1]:
# PACKAGE IMPORTS
import gc  # Garbage collection module
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import subprocess


In [2]:
# DEFINITIONS

# General Definitions

# Pick the data type
DATA_TYPE = "energy"
#DATA_TYPE = "water"

# Output debugging and testing info
VERBOSE = False
VERBOSE = True

# Show intermediate charts
SHOW_CHARTS = True
SHOW_CHARTS = False

# Get username for path definitions
# Run the `whoami` command and store username
username = subprocess.run(["whoami"], capture_output=True, text=True).stdout.strip()

# Input data path
datapath="/Users/"+username+"/Google Drive/Shared drives/Sustainability Team/[02] Analytics & Insights/FEM Analytics and Insights/FEM24 Checks Outlier Analysis/Data/Modified IQR/"


# Data Handling Definitions

# Define bin width for histograms and modified IQR analysis
energy_bin_width = 1e6
water_bin_width = 1e6

energy_per_unit_bin_width = 10
water_per_unit_bin_width = 10

energy_yoy_bin_width = 1
water_yoy_bin_width = 1

# Modified IQR Analysis
# percentages for Q1, Q2, Q3
quartiles = [0.25, 0.50, 0.75]


# Assign variables based on data type selection
if DATA_TYPE == "energy":
    iqr_step = 1.5 # Modified IQR step size

    dataname_all = "total_energy_all_data_FEM23.csv"
    dataname_per = "total_energy_per_unit_FEM23.csv"
    dataname_yoy = "total_energy_yoy_23_22.csv"
    
    response_units = "mj"
    fac_suffix = "_total_"+response_units
    
    all_bin_width = energy_bin_width
    per_unit_bin_width = energy_per_unit_bin_width
    yoy_bin_width = energy_yoy_bin_width
elif DATA_TYPE == "water":
    iqr_step = 1.7 # Modified IQR step size

    dataname_all = "total_water_all_data_FEM23.csv"
    dataname_per = "total_water_per_unit_FEM23.csv"
    dataname_yoy = "total_water_yoy_22_21.csv"

    response_units = "l"
    fac_suffix = "_total_water_"+response_units

    all_bin_width = water_bin_width
    per_unit_bin_width = water_per_unit_bin_width
    yoy_bin_width = water_yoy_bin_width

    
# Define response column name to use in each dataframe
response_all = "response_"+response_units  # input data df_all
response_sum = "response_sum"              # df_sum, calculated from df_all
response_per = response_units+"_per_unit"  # input data df_per
response_yoy = "change%"                   # input data df_yoy (year on year)
response_yos = "change%"                  # df_yos (year on year summed), calculated from df_yoy


In [3]:
# IMPORT DATA

df_all = pd.read_csv(datapath+dataname_all, delimiter=",")
df_per = pd.read_csv(datapath+dataname_per, delimiter=",")
df_yoy = pd.read_csv(datapath+dataname_yoy, delimiter=",")

if VERBOSE:
    display(df_all.head())
    display(df_per.head())
    display(df_yoy.head())

    print(f"length(df_all): {len(df_all)}")
    print(f"length(df_per): {len(df_per)}")
    print(f"length(df_yoy): {len(df_yoy)}")


,assessment_id,account_id,rfi_pid,status,facility_posted,refid,response,refid_2
0,femsurvey:337325c0-0de2-4aa3-8d9e-79105e537e84,5a555c52d41b5d12cba674c7,fem2023,VRF,True,vehicle_total_mj,2469014.0,vehicle_total_mj
1,femsurvey:829a14d3-f506-4fc4-b76e-9a78b040db1a,51af1301aad8beff4fbd2a051f91f6f6,fem2023,VRF,True,vehicle_total_mj,2371698.0,vehicle_total_mj
2,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,bbf1677f4621395211a6d61f062cf6fa,fem2023,VRF,True,materialProduction_total_mj,229000000.0,materialProduction_total_mj
3,femsurvey:442b57b5-2678-412f-be7b-fb6798d4b1dd,5a1f5ee898d35f0486f6ebad,fem2023,VRF,True,domestic_total_mj,1587031.0,domestic_total_mj
4,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,60078743b8c28e000d9844cc,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,5491908.0,printingProductDyeingAndLaundering_total_mj


,assessment_id,refid_mj,facility_type,response_mj,refid_prod_vol,response_prod_vol,mj_per_unit,facility_type_2
0,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,materialProduction_total_mj,materialProduction,229000000.0,materialProductiontotal,1770621.0,129.076400,materialProduction
1,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,5491908.0,printingProductDyeingAndLaunderingtotal,1310650.0,4.190217,printingProductDyeingAndLaundering
2,femsurvey:2dfd27ff-a43e-4697-a13c-6bd05edf7dfd,finalProductAssembly_total_mj,finalProductAssembly,7896975.0,finalProductAssemblytotal,5304.0,1488.872000,finalProductAssembly
3,femsurvey:ea131d14-15e5-4b7f-af64-6a4f60af4c50,finalProductAssembly_total_mj,finalProductAssembly,3855600.0,finalProductAssemblytotal,445650.0,8.651632,finalProductAssembly
4,femsurvey:9b195bb0-cd20-44ca-ae37-127e1e5e0d7d,finalProductAssembly_total_mj,finalProductAssembly,3903322.0,finalProductAssemblytotal,400000.0,9.758304,finalProductAssembly


,account_id,refid,2023response,2022response,change,change%
0,5a6e9b5b963a52483a65a198,ensourcedieseltotal,2.829653e+04,3.810625e+04,-9.809720e+03,-0.25743
1,5a6e9b5b963a52483a65a198,ensourceelectricpurchtotal,3.086748e+05,3.466800e+05,-3.800520e+04,-0.10963
2,5a77fba4d02d57577cfc1f11,ensourcedieseltotal,3.981732e+06,4.726444e+06,-7.447130e+05,-0.15756
3,5a77fba4d02d57577cfc1f11,ensourceelectricpurchtotal,1.737634e+06,5.950800e+05,1.142554e+06,1.92000
4,5a77fba4d02d57577cfc1f11,ensourcenaturalgastotal,2.620000e+09,2.840000e+09,-2.100000e+08,-0.07442


length(df_all): 24419
length(df_per): 12911
length(df_yoy): 22182


In [4]:
def append_matched_column(df_source, df_to_update, column_to_check, column_to_compare, DATA_TYPE):
    """
    Append a column from one DataFrame to another based on matching values in a specified column.

    Args:
        df_source (pd.DataFrame): The source DataFrame containing the reference data.
        df_to_update (pd.DataFrame): The target DataFrame where the new column will be added.
        column_to_check (str): The column used to match rows between the two DataFrames.
        column_to_compare (str): The column in the source DataFrame whose values will be added to the target DataFrame.
        DATA_TYPE (str): A descriptive label of the dataset for logging purposes.

    Returns:
        pd.DataFrame: The updated target DataFrame with the added column.
    """
    # Identify rows in the source DataFrame that have non-unique values in the specified column
    non_unique = df_source[df_source.duplicated(subset=[column_to_check], keep=False)]
    
    # Identify groups of rows where the same value in column_to_check maps to multiple unique values in column_to_compare
    groups_with_different_values = non_unique.groupby(column_to_check).filter(
        lambda x: x[column_to_compare].nunique() > 1
    )
    
    # Display mismatched values if any exist; otherwise, print a success message
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        if len(groups_with_different_values) > 0:
            display(groups_with_different_values.sort_values(by=column_to_check))
        else:
            print(f"*** No mismatched {column_to_compare} values among unique {column_to_check} values for {DATA_TYPE}\n")
    
    # Create a DataFrame containing unique pairs of values from column_to_check and column_to_compare
    df_stat_uniq = df_source[[column_to_check, column_to_compare]].drop_duplicates(subset=[column_to_check])
    
    # Create a mapping from the unique values in column_to_check to their corresponding values in column_to_compare
    mapping = df_stat_uniq.set_index(column_to_check)[column_to_compare]
    
    # Add a new column to the target DataFrame by mapping values from the source DataFrame
    # Matches are based on the values in column_to_check
    df_to_update[column_to_compare] = df_to_update[column_to_check].map(mapping)
    
    return df_to_update


In [5]:
# Clean and process df_all

# Rename columns for consistency (df_per is the reference)
df_all.rename(columns={"refid": "refid_"+response_units}, inplace=True)
df_all.rename(columns={"response": response_all}, inplace=True)


# Add facility_type columns
#     facility_type = strip units from refid
#     facility_type_2 = combine rawMaterial* categories from facility_type
A = "rawMaterialProcessing"
B = "rawMaterialCollection"
C = "rawMaterialProcessing_Collection"

df_all["facility_type"] = df_all["refid_"+response_units].str.rstrip(fac_suffix)
df_all["facility_type_2"] = df_all["facility_type"].replace({A: C, B: C})


In [6]:

display(df_all[df_all["facility_type"].isin([A,B])])


,assessment_id,account_id,rfi_pid,status,facility_posted,refid_mj,response_mj,refid_2,facility_type,facility_type_2
9,femsurvey:752eb819-7c4c-4513-8530-4ca7526f67ef,646dfe749e5a16000cb740b0,fem2023,ASC,True,rawMaterialProcessing_total_mj,15014387.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
174,femsurvey:e4f1c0f4-683b-46fe-b893-c81bca6f3100,645b6a0525f4d8000cd8cc55,fem2023,VRF,True,rawMaterialProcessing_total_mj,2968811.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
192,femsurvey:388884e3-1b31-4f24-a546-f297da64a411,0f95596cfb33b1171106a7a079efe055,fem2023,ASC,True,rawMaterialProcessing_total_mj,7374251.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
211,femsurvey:6989f615-9234-405b-a6c8-abbf87a1e926,62b1ae81579380000d68dce0,fem2023,VRF,True,rawMaterialCollection_total_mj,8834989.0,rawMaterialProcessing_Collection,rawMaterialCollection,rawMaterialProcessing_Collection
223,femsurvey:3839a1b8-11ef-408f-9d19-6a2ac17d5be1,66ffe16b9ed8a4000c37791b,fem2023,VRF,True,rawMaterialProcessing_total_mj,11087385.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
...,...,...,...,...,...,...,...,...,...,...
24108,femsurvey:4a6b77c0-c663-47f3-af81-0ebcdc731c80,6721b62bebeeed000c34f1e8,fem2023,ASC,True,rawMaterialProcessing_total_mj,2449830.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
24244,femsurvey:a67d586b-74ac-4a4f-aac7-a338d5d475af,5a97bc96f28aaa21d5d4999b,fem2023,VRF,True,rawMaterialProcessing_total_mj,2063974.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
24304,femsurvey:82d22309-9632-44ff-bba1-baff74f12234,63a29a88a67111000bbcb709,fem2023,VRF,True,rawMaterialProcessing_total_mj,137000000.0,rawMaterialProcessing_Collection,rawMaterialProcessing,rawMaterialProcessing_Collection
24344,femsurvey:7e6cb4bd-04d5-4aeb-ac45-70fd63ed211d,646445c9ed2d47000cd529bc,fem2023,VRF,True,rawMaterialCollection_total_mj,18874709.0,rawMaterialProcessing_Collection,rawMaterialCollection,rawMaterialProcessing_Collection


In [7]:
# Clean and process df_sum

# This is a new dataframe containing assessment_id, account_id, and 
# the sum of response from df_all grouped by assessment_id

# Group by column A and calculate the sum of column B
A = "assessment_id"
B = response_all
C = "assessmentResponseSum"

df_sum = df_all.groupby(A).agg(
    account_id = ("account_id", "first"),  # Take the first value from the column 
    status = ("status", "first"),          # this is needed to make the IQR routine work
    response_sum = (B, "sum"),             # Sum of column B
).reset_index()
df_sum["facility_type_2"] = C  # this is needed to make the IQR routine work


In [8]:
# Clean and process df_per

# Add status from df_all (by assessment_id) to df_per
column_to_check = "assessment_id"
column_to_compare = "status"
df_per = append_matched_column(df_all, df_per, column_to_check, column_to_compare, DATA_TYPE)

# Add account_id from df_all (by assessment_id) to df_per
column_to_check = "assessment_id"
column_to_compare = "account_id"
df_per = append_matched_column(df_all, df_per, column_to_check, column_to_compare, DATA_TYPE)


*** No mismatched status values among unique assessment_id values for energy

*** No mismatched account_id values among unique assessment_id values for energy



In [9]:
# Clean and process df_yoy

# Add status from df_all (by account_id) to df_yoy
column_to_check = "account_id"
column_to_compare = "status"
df_yoy = append_matched_column(df_all, df_yoy, column_to_check, column_to_compare, DATA_TYPE)

# Convert % column to percentages instead of fractions
df_yoy["change%"] = df_yoy["change%"]*100.0

# Create "fake" facility column 
# (cuz we don't want to segment by refid but need a facility type for consistency)
df_yoy["facility_placeholder"] = "yoy_facility_placeholder"


*** No mismatched status values among unique account_id values for energy



In [10]:
def create_yos(df, groupby_column, status_column, facility_column, response1_column, response0_column, 
               response1_output, response0_output, abs_change_output, pct_change_output):
    """
    Creates a summary DataFrame that calculates year-over-year (YoY) changes in responses.

    Args:
        df (pd.DataFrame): The input DataFrame containing the data.
        groupby_column (str): The column to group by (e.g., account ID).
        status_column (str): The column indicating the status (used for additional calculations).
        facility_column (str): The column indicating the facility (used for additional calculations).
        response1_column (str): The column containing the current year's response counts.
        response0_column (str): The column containing the previous year's response counts.
        response1_output (str): The desired output column name for the current year's response.
        response0_output (str): The desired output column name for the previous year's response.
        abs_change_output (str): The desired output column name for absolute change in responses.
        pct_change_output (str): The desired output column name for percentage change in responses.

    Returns:
        pd.DataFrame: A new DataFrame with grouped data, YoY response counts, and change calculations.
    """
    # Group the DataFrame by the specified column and aggregate relevant fields
    df_yos = df.groupby(groupby_column).agg(
        # Retain the first value of the status column for each group (used for additional routines)
        status=(status_column, "first"),
        # Retain the first value of the facility column for each group (used for additional routines)
        facility_placeholder=(facility_column, "first"),
        # Sum the response counts for the current year
        resp_col1=(response1_column, "sum"),
        # Sum the response counts for the previous year
        resp_col0=(response0_column, "sum"),
    ).reset_index()

    # Rename the aggregated response columns to the specified output column names
    df_yos.rename(columns={
        "resp_col1": response1_output,
        "resp_col0": response0_output,
    }, inplace=True)

    # Calculate the absolute change in response counts between years
    df_yos[abs_change_output] = df_yos[response1_output] - df_yos[response0_output]

    # Calculate the percentage change in response counts between years
    # Avoid division by zero by ensuring response0_output is not zero
    df_yos[pct_change_output] = 100 * df_yos[abs_change_output] / df_yos[response0_output]

    return df_yos


In [11]:
# Clean and process df_yos

# This is a new dataframe based on df_yoy containing account_id, status, 
# facility_placeholder, the sums (over refid per account_id) of 2023response and
# 2022response, and change and change% values based on those summed responses

# Define column names for creating df_yos
groupby_column = "account_id"
status_column = "status"
facility_column = "facility_placeholder"
abs_change_output = "change"
pct_change_output = "change%"
    
if DATA_TYPE == "energy":
    response1_column = "2023response"
    response0_column = "2022response"
    response1_output = "ResponseSum2023"
    response0_output = "ResponseSum2022"
elif DATA_TYPE == "water":
    response1_column = "2022response"
    response0_column = "2021response"
    response1_output = "ResponseSum2022"
    response0_output = "ResponseSum2021"

df_yos = create_yos(df_yoy, groupby_column, status_column, facility_column, response1_column, response0_column, response1_output, response0_output, abs_change_output, pct_change_output)


In [12]:
# Display info about input dataframes
if VERBOSE:
    print("********* df_all")
    display(df_all.head())
    print("********* df_sum")
    display(df_sum.head())
    print("********* df_per")
    display(df_per.head())
    print("********* df_yoy")
    display(df_yoy.head())
    print("********* df_yos")
    display(df_yos.head())

    print(f"length(df_all): {len(df_all)}")
    print(f"length(df_sum): {len(df_sum)}")
    print(f"length(df_per): {len(df_per)}")
    print(f"length(df_yoy): {len(df_yoy)}")
    print(f"length(df_yos): {len(df_yos)}")


********* df_all


,assessment_id,account_id,rfi_pid,status,facility_posted,refid_mj,response_mj,refid_2,facility_type,facility_type_2
0,femsurvey:337325c0-0de2-4aa3-8d9e-79105e537e84,5a555c52d41b5d12cba674c7,fem2023,VRF,True,vehicle_total_mj,2469014.0,vehicle_total_mj,vehicle,vehicle
1,femsurvey:829a14d3-f506-4fc4-b76e-9a78b040db1a,51af1301aad8beff4fbd2a051f91f6f6,fem2023,VRF,True,vehicle_total_mj,2371698.0,vehicle_total_mj,vehicle,vehicle
2,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,bbf1677f4621395211a6d61f062cf6fa,fem2023,VRF,True,materialProduction_total_mj,229000000.0,materialProduction_total_mj,materialProduction,materialProduction
3,femsurvey:442b57b5-2678-412f-be7b-fb6798d4b1dd,5a1f5ee898d35f0486f6ebad,fem2023,VRF,True,domestic_total_mj,1587031.0,domestic_total_mj,domestic,domestic
4,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,60078743b8c28e000d9844cc,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,5491908.0,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering


********* df_sum


,assessment_id,account_id,status,response_sum,facility_type_2
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,3.369713e+05,assessmentResponseSum
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,9.289080e+05,assessmentResponseSum
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,2.633258e+09,assessmentResponseSum
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,7.277440e+07,assessmentResponseSum
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,4.626482e+06,assessmentResponseSum


********* df_per


...

********* df_yoy


,account_id,refid,2023response,2022response,change,change%,status,facility_placeholder
0,5a6e9b5b963a52483a65a198,ensourcedieseltotal,2.829653e+04,3.810625e+04,-9.809720e+03,-25.743,VRF,yoy_facility_placeholder
1,5a6e9b5b963a52483a65a198,ensourceelectricpurchtotal,3.086748e+05,3.466800e+05,-3.800520e+04,-10.963,VRF,yoy_facility_placeholder
2,5a77fba4d02d57577cfc1f11,ensourcedieseltotal,3.981732e+06,4.726444e+06,-7.447130e+05,-15.756,VRF,yoy_facility_placeholder
3,5a77fba4d02d57577cfc1f11,ensourceelectricpurchtotal,1.737634e+06,5.950800e+05,1.142554e+06,192.000,VRF,yoy_facility_placeholder
4,5a77fba4d02d57577cfc1f11,ensourcenaturalgastotal,2.620000e+09,2.840000e+09,-2.100000e+08,-7.442,VRF,yoy_facility_placeholder


********* df_yos


,account_id,status,facility_placeholder,ResponseSum2023,ResponseSum2022,change,change%
0,0f95596cfb33b1171106a7a07900502c,VRF,yoy_facility_placeholder,20887795.90,17915367.20,2972428.70,16.591503
1,0f95596cfb33b1171106a7a079005076,VRF,yoy_facility_placeholder,40329335.10,10763363.90,29565971.20,274.690807
2,0f95596cfb33b1171106a7a079005c2b,VRF,yoy_facility_placeholder,17711762.50,20499249.10,-2787486.60,-13.597994
3,0f95596cfb33b1171106a7a07900cd0f,ASC,yoy_facility_placeholder,844581.60,718470.00,126111.60,17.552800
4,0f95596cfb33b1171106a7a07900d27d,ASC,yoy_facility_placeholder,1014222.14,1210535.26,-196313.12,-16.217051


length(df_all): 24419
length(df_sum): 11709
length(df_per): 12911
length(df_yoy): 22182
length(df_yos): 9110


In [13]:
if VERBOSE:
    print("*** Comparing contents of df_all and df_per\n")

    # Filter rows in df2 where values in column A are not in column A of df1
    df_diff = df_per[~df_per["assessment_id"].isin(df_all["assessment_id"])]
    # Display the result
    print("This dataframe contains assessment_id values that are in df_per but not in df_all.")
    print("It should be empty.")
    display(df_diff)

    # Filter rows in df2 where values in column A are not in column A of df1
    df_diff = df_all[~df_all["assessment_id"].isin(df_per["assessment_id"])]
    print("\nThis dataframe contains assessment_id values that are in df_all but not in df_per.")
    print("It should NOT be empty.")
    # Display the result
    display(df_diff)
    

*** Comparing contents of df_all and df_per

This dataframe contains assessment_id values that are in df_per but not in df_all.
It should be empty.


,assessment_id,refid_mj,facility_type,response_mj,refid_prod_vol,response_prod_vol,mj_per_unit,facility_type_2,status,account_id



This dataframe contains assessment_id values that are in df_all but not in df_per.
It should NOT be empty.


,assessment_id,account_id,rfi_pid,status,facility_posted,refid_mj,response_mj,refid_2,facility_type,facility_type_2
10,femsurvey:1f7bd089-dbf2-4d08-ab9d-2e3b934a6061,5f758029ca8234000a181853,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,5484571.00,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering
54,femsurvey:9e0e6fbd-c9e3-49b1-8be5-0946016efa13,66157691a2a150000c919cb6,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,2105212.00,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering
64,femsurvey:0b9b9001-11a1-44c6-bf12-a0d2a45c1ef5,5b13bc9ae7e8e430559ce515,fem2023,VRF,True,domestic_total_mj,213120.00,domestic_total_mj,domestic,domestic
194,femsurvey:9191994b-9ecd-4bb4-911b-9bc2bb28b6b6,65d315d734fc15000c5d147d,fem2023,VRF,True,vehicle_total_mj,45450.59,vehicle_total_mj,vehicle,vehicle
201,femsurvey:dd8cac70-e0da-4802-9664-1b5bebf9d999,63c6d2a3dcdb66000bb97cf3,fem2023,VRC,True,printingProductDyeingAndLaundering_total_mj,37265810.00,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering
...,...,...,...,...,...,...,...,...,...,...
24117,femsurvey:611f2509-3f0d-4ec1-80c2-781fa587f8da,660ced3a794319000c03a26f,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,442478.10,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering
24166,femsurvey:db28eb72-2640-4f75-bde8-5ba8d9e726b3,604a089914aea2000aebe8ff,fem2023,VRF,True,domestic_total_mj,6406141.00,domestic_total_mj,domestic,domestic
24210,femsurvey:aa1466ac-5126-4754-945e-26b0b6e601e7,606443de4ba279000b394d4a,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,47104324.00,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering
24241,femsurvey:3bfa4569-2ca9-47a7-94c6-d52a394d077a,6038badde0516f000bcba0dd,fem2023,VRF,True,hardComponentTrimProduction_total_mj,784735.00,hardComponentTrimProduction_total_mj,hardComponentTrimProduction,hardComponentTrimProduction


In [14]:
# CREATE FILTERED DF

def filter_df(df, fac_column, facility_type, status_type):
    """
    Filters a DataFrame based on specified facility type and status type conditions.

    Args:
        df (pd.DataFrame): The input DataFrame to filter.
        fac_column (str): The column name in the DataFrame representing the facility type.
        facility_type (str or list): The facility type(s) to filter by. Can be a single value or a list of values.
        status_type (str): The status type to filter by. Use "all" to include all statuses.

    Returns:
        pd.DataFrame: A filtered copy of the input DataFrame that meets the specified conditions.
    """
    # Create a condition to filter rows based on the facility type
    # If `facility_type` is a list, check if the values in `fac_column` are in the list
    # Otherwise, check for equality with a single value
    if isinstance(facility_type, list):
        fac_condition = df[fac_column].isin(facility_type)
    else:
        fac_condition = df[fac_column] == facility_type

    # Create a condition to filter rows based on the status type
    # If `status_type` is "all", include all rows (condition is always True)
    # Otherwise, filter rows where the `status` column matches the given `status_type`
    if status_type == "all":
        status_condition = True
    else:
        status_condition = df["status"] == status_type

    # Combine the facility and status conditions to filter the DataFrame
    filtered_df = df[fac_condition & status_condition].copy()

    return filtered_df
    

In [15]:
# SORT FILTERED DF INTO BINS
    
def create_bins(filtered_df, response, bin_width, DO_CHUNK=False, chunk_size=1_000_000):
    """
    Creates bins and computes the histogram of a response column in a DataFrame.

    Args:
        filtered_df (pd.DataFrame): The input DataFrame containing the data to bin.
        response (str): The column name in the DataFrame to be binned.
        bin_width (float): The width of each bin.
        DO_CHUNK (bool): Whether to process the data in chunks (useful for very large DataFrames).
                         Defaults to False for processing the entire DataFrame at once.
        chunk_size: Adjust the chunk size based on available system memory.

    Returns:
        tuple: A tuple containing:
            - bins (np.ndarray): The bin edges.
            - bin_counts (np.ndarray): The counts of data points in each bin.
            - bin_edges (np.ndarray): The same as bins, included for consistency with `np.histogram`.
    """
    # Calculate the minimum and maximum values of the response column
    # Ensure bins start at 0 or below, with a minimum of response_min
    response_min = min(filtered_df[response].min(), 0)
    response_max = filtered_df[response].max()
    
    # Determine the start point and end point for the bins
    bin_start = response_min
    bin_end = np.ceil(response_max / bin_width) * bin_width  # Round up to the nearest bin width

    # Generate an array of bin edges based on the calculated start, end, and bin width
    bins = np.arange(bin_start, bin_end + bin_width, bin_width)

    # Compute bin counts and edges based on the CHUNK flag
    if not DO_CHUNK:
        # If DO_CHUNK is False, compute the histogram for the entire DataFrame
        bin_counts, bin_edges = np.histogram(filtered_df[response].to_numpy(), bins=bins)
    else:
        # If DO_CHUNK is True, process the DataFrame in chunks to handle large datasets
        bin_counts = np.zeros(len(bins) - 1, dtype=int)  # Pre-allocate space for bin counts

        # Process data in chunks, summing the counts for each bin
        for i in range(0, len(filtered_df), chunk_size):
            chunk = filtered_df[response].iloc[i:i+chunk_size].to_numpy()
            chunk_counts, _ = np.histogram(chunk, bins=bins)
            bin_counts += chunk_counts  # Accumulate bin counts from each chunk
        bin_edges = bins  # Bin edges remain unchanged

    return bins, bin_counts, bin_edges


In [16]:
# APPLY A TRANSFORMATION TO THE BIN COUNTS

def transform_bin_counts(bin_counts):
    return np.sqrt(bin_counts) # square root transform


In [17]:
# DISPLAY A HISTOGRAM CHART

def hist_plot(bin_counts_plot, bin_edges, bin_width, xmax=np.nan, mark_vert=0) -> None:
    """
    Plots a histogram with transformed bin counts and log-scaled y-axis.

    Args:
        bin_counts_plot (array-like): The counts for each bin (e.g., transformed or raw).
        bin_edges (array-like): The edges of the bins.
        bin_width (float): The width of each bin.
        xmax (float, optional): The maximum value for the x-axis. 
                                Defaults to 0, which uses the maximum value of `bin_edges`.
        mark_vert (float, optional): A vertical line to mark a threshold (e.g., outlier cutoff). 
                                     Defaults to 0 (no line is drawn).

    Returns:
        None
    """
    # Plot the histogram as a bar chart with specified bin counts and edges
    plt.bar(
        bin_edges[:-1],  # Start points of each bin
        bin_counts_plot,  # Heights of the bars (bin counts)
        width=bin_width,  # Width of each bar
        edgecolor="black",  # Outline color of bars
        color="skyblue",  # Fill color of bars
        alpha=0.7  # Transparency level for the bars
    )
    
    # Add axis labels for clarity
    plt.xlabel("Response Binned Ranges")
    plt.ylabel("(Transformed) Counts")
    
    # Use a logarithmic scale for the y-axis to better visualize wide-ranging counts
    plt.yscale("log")
    
    # Set the range for the x-axis
    # If `xmax` is specified (greater than 0), limit the x-axis to that value
    # Otherwise, use the full range of `bin_edges`
    if np.isnan(xmax):
        plt.xlim(bin_edges.min(), bin_edges.max())
    else:
        plt.xlim(bin_edges.min(), xmax)

    # Optionally mark a vertical line at a specified x-value (`mark_vert`)
    # Useful for indicating thresholds, such as an outlier cutoff
    if mark_vert != 0:
        plt.axvline(
            mark_vert,  # x-coordinate of the vertical line
            color="blue",  # Line color
            linestyle="--",  # Dashed line style
            linewidth=1,  # Line thickness
            label=f"x = {mark_vert/1e6}M"  # Label showing the marked value in millions
        )
    
    # Add a grid to the plot for improved readability
    plt.grid(True)
    
    # Show the plot
    plt.show()
    

In [18]:
def create_bins_df(bin_edges, bin_counts, bin_counts_transf):    
    """
    Creates a DataFrame to represent histogram bins and their associated counts.

    Args:
        bin_edges (array-like): The edges of the bins. Should have a length of `len(bin_counts) + 1`.
        bin_counts (array-like): The counts of data points in each bin.
        bin_counts_transf (array-like): The transformed counts of data points in each bin. 
                                         Should have the same length as `bin_counts`.

    Returns:
        pd.DataFrame: A DataFrame with the following columns:
            - "bin_start": The start of each bin (excluding the last edge).
            - "bin_end": The end of each bin (excluding the first edge).
            - "bin_counts": The raw counts for each bin.
            - "bin_counts_transf": The transformed counts for each bin.

    Raises:
        ValueError: If the input arrays have inconsistent lengths.
    """
    # Validate input lengths to ensure consistency
    # `bin_edges` should have one more element than `bin_counts`, and `bin_counts` should match `bin_counts_transf` in length
    if len(bin_edges) != len(bin_counts) + 1 or len(bin_counts) != len(bin_counts_transf):
        raise ValueError("create_bins_df: Input arrays have inconsistent lengths.")

    # Construct the DataFrame with bin information
    return pd.DataFrame({
        "bin_start": bin_edges[:-1],  # Start of each bin (exclude the last edge)
        "bin_end": bin_edges[1:],    # End of each bin (exclude the first edge)
        "bin_counts": bin_counts,    # Raw counts in each bin
        "bin_counts_transf": bin_counts_transf  # Transformed counts in each bin
    }, copy=False)  # Avoid unnecessary data copying for better performance


In [19]:
# REMOVE ROWS WHERE bin_counts IS ZERO IN SPECIFIED COLUMN

def remove_zero_bins(df_bins, colname):
    return df_bins[df_bins[colname] > 0]
    

In [20]:
# SORT BINS INTO QUARTILES BASED ON (TRANSFORMED) BIN COUNTS

def quartile_sort(df_bins, quartiles):    
    """
    Sorts bins into quartiles (or specified percentiles) based on transformed cumulative sums.

    Args:
        df_bins (pd.DataFrame): A DataFrame containing bin information, including columns for:
            - "bin_counts": The raw counts for each bin.
            - "bin_counts_transf": The transformed counts for each bin.
        quartiles (list of float): A list of quartile or percentile values (e.g., [0.25, 0.5, 0.75]).

    Returns:
        tuple: A tuple containing:
            - bin_end_quartiles (list): The bin end values where the cumulative sums reach each quartile.
            - bin_reached (pd.Series): The bin corresponding to the last calculated quartile.

    Notes:
        Quartile values must be between 0 and 1, representing percentages (e.g., 0.25 for 25%).
    """
    # Compute cumulative sums for both raw and transformed bin counts
    df_bins["cumulative_sum"] = df_bins["bin_counts"].cumsum()
    df_bins["transf_cumulative_sum"] = df_bins["bin_counts_transf"].cumsum()

    # This block is not currently used but left here just in case...
    # Compute reverse cumulative sums (from the end of the bins backward)
    #df_bins["reverse_cumulative_sum"] = df_bins["bin_counts"][::-1].cumsum()[::-1]
    #df_bins["reverse_transf_cumulative_sum"] = df_bins["bin_counts_transf"][::-1].cumsum()[::-1]
    
    bin_end_quartiles = []  # Store the bin end values for each quartile

    # Iterate over each specified quartile
    for i in quartiles:
        print(f"Quartile: {i}")
        
        # Calculate the target cumulative sum for the current quartile
        target_value = df_bins["bin_counts_transf"].sum() * i
        
        # Find the first bin where the transformed cumulative sum meets or exceeds the target
        bin_reached = df_bins[df_bins["transf_cumulative_sum"] >= target_value].iloc[0]

        # Display information about the bin and its corresponding quartile
        print(f"The bin in which the (transformed) cumulative sum reaches {target_value}: {bin_reached["bin_end"]} ({bin_reached["bin_end"]/1e6}M)\n")
    
        # Store the bin end value corresponding to the quartile
        bin_end_quartiles.append(bin_reached["bin_end"])

    return bin_end_quartiles, bin_reached
    

In [21]:
# PERFORM MODIFIED IQR METHOD

def modified_iqr(bin_end_quartiles, iqr_direction, iqr_step=1.5):
    """
    Computes the Interquartile Range (IQR) and returns a modified threshold based on the direction and step multiplier.

    Args:
        bin_end_quartiles (list of float): A list of bin end values corresponding to quartiles.
                                           Expected to contain at least the 1st (25%) and 3rd (75%) quartiles.
        iqr_direction (str): The direction for the modified IQR threshold, either "UPPER" or "LOWER".
        iqr_step (float, optional): The step multiplier for the IQR to calculate the threshold. 
                                    Defaults to 1.5.

    Returns:
        tuple: A tuple containing:
            - iqr (float): The computed IQR value (Q3 - Q1).
            - iqr_threshold (float): The threshold based on the specified direction and step multiplier.

    Raises:
        ValueError: If `iqr_direction` is not "UPPER" or "LOWER".
    """
    # Compute the IQR as the difference between the 3rd (75%) and 1st (25%) quartile
    iqr = bin_end_quartiles[2] - bin_end_quartiles[0]

    # Calculate the modified IQR threshold based on the direction
    if iqr_direction.upper() == "UPPER":
        iqr_threshold = bin_end_quartiles[2] + iqr_step * iqr
    elif iqr_direction.upper() == "LOWER":
        iqr_threshold = bin_end_quartiles[0] - iqr_step * iqr
    else:
        raise ValueError("modified_iqr: Invalid value for `iqr_direction`. Must be 'UPPER' or 'LOWER'.")

    return iqr, iqr_threshold
    

In [22]:
# FIND BIN FOR IQR_THRESHOLD

def find_iqr_threshold_bin(df_bins, target_value, iqr_direction):    
    """
    Identifies the bin corresponding to the IQR threshold value in the specified direction 
    (either "UPPER" or "LOWER") based on cumulative bin counts.

    Args:
        df_bins (pd.DataFrame): A DataFrame containing bin information, including columns for:
            - "bin_end": The end value for each bin.
            - "cumulative_sum": The cumulative sum of bin counts.
        target_value (float): The target value for the IQR threshold.
        iqr_direction (str): The direction for the IQR threshold, either "UPPER" or "LOWER".

    Returns:
        tuple: A tuple containing:
            - bin_reached (float): The bin end value where the target cumulative sum is reached or exceeded.
            - bin_reached_total (int): The cumulative sum at the identified bin.

    Notes:
        - If the target value exceeds the range of `bin_end` values, the function returns the maximum or minimum bin end value and a total of 0.
    """
    # Handle the UPPER direction for identifying the IQR threshold bin
    if iqr_direction.upper() == "UPPER":
        if target_value > df_bins["bin_end"].max():
            # If target exceeds the max bin value, return max bin and 0 total
            bin_reached = df_bins["bin_end"].max()
            bin_reached_total = 0
        else:
            # Find the first bin where the bin end is greater than or equal to the target value
            bin_reached_df = df_bins[df_bins["bin_end"] >= target_value].iloc[0]
            bin_reached = bin_reached_df["bin_end"]
            bin_reached_total = bin_reached_df["cumulative_sum"]

    # Handle the LOWER direction for identifying the IQR threshold bin
    elif iqr_direction.upper() == "LOWER":
        if target_value < df_bins["bin_end"].min():
            # If target is less than the min bin value, return min bin and 0 total
            bin_reached = df_bins["bin_end"].min()
            bin_reached_total = 0
        else:
            # Find the last bin where the bin end is less than or equal to the target value
            bin_reached_df = df_bins[df_bins["bin_end"] <= target_value].iloc[-1]
            bin_reached = bin_reached_df["bin_end"]
            bin_reached_total = bin_reached_df["cumulative_sum"]

    return bin_reached, bin_reached_total
    

In [23]:
# Define facility type categories from those present in data

def define_facility_types(df, fac_column):
    """
    Extracts the unique facility types from a specified column in the given DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame containing facility information.
        fac_column (str): The name of the column containing facility type values.

    Returns:
        list: A list of unique facility types present in the specified column.
    """
    # Extract unique facility types using set for faster lookup and conversion to list
    facility_types = list(set(df[fac_column]))

    return facility_types
    

In [24]:
# SET UP DEFINITIONS FOR DF HANDLING

# Note: df_yos is in this list twice because we do determine an upper and lower threshold
# df names for logging and identification
df_list_names = ["df_all", "df_sum", "df_per", "df_yoy", "df_yos", "df_yos"]
# dfs for iteration
df_list = [df_all, df_sum, df_per, df_yoy, df_yos, df_yos]  

summary_list = []  # Stores IQR analysis results for each DataFrame and facility type

# Define configurations in a dictionary (indices refer to df_list)
configurations = {
    0: {
        "response": response_all, 
        "bin_width": all_bin_width, 
        "fac_column": "facility_type_2", 
        "iqr_direction": "upper"
    },
    1: {
        "response": response_sum, 
        "bin_width": all_bin_width, 
        "fac_column": "facility_type_2", 
        "iqr_direction": "upper"
    },
    2: {
        "response": response_per, 
        "bin_width": per_unit_bin_width, 
        "fac_column": "facility_type_2", 
        "iqr_direction": "upper"
    },
    3: {
        "response": response_yoy,
        "bin_width": yoy_bin_width,
        "fac_column": "facility_placeholder",
        "iqr_direction": "upper",
    },
    4: {
        "response": response_yos,
        "bin_width": yoy_bin_width,
        "fac_column": "facility_placeholder",
        "iqr_direction": "upper",
    },
    5: {
        "response": response_yos,
        "bin_width": yoy_bin_width,
        "fac_column": "facility_placeholder",
        "iqr_direction": "lower",
    }
}


In [39]:
# Calculate sigma assuming:
# The distribution is a one-sided symmetric distirbution with mean = 0

def calc_sigma(df):
    # compute the mean of the one-sided data
    one_side_mean = df[response].mean()
    #one_side_mean = np.sqrt(df[response]).mean()
    #one_side_mean = np.log10(df[response]).mean()
    # compute the standard deviation of the one-sided data
    one_side_sigma = df[response].std()
    #one_side_sigma = np.sqrt(df[response]).std()
    #one_side_sigma = np.log10(df[response]).std()
    # recover the original sigma
    full_sigma_v1 = one_side_mean * np.sqrt(np.pi) / np.sqrt(2)
    full_sigma_v2 = one_side_sigma / (np.sqrt(1 - (2/np.pi)))
    
    print(one_side_mean, one_side_sigma)
    
    return full_sigma_v1, full_sigma_v2
    

In [40]:
# RUN IQR ANALYSIS USING FUNCTIONS DEFINED ABOVE

summary_row_dict = {} # Initialize summary results dicitonary
summary_list = [] # Initialize summary results row list

# Iterate through each DataFrame in the list
for i, df_original in enumerate(df_list):
    if VERBOSE:
        print("=================================================================================\n")
    print(f"********* Starting df iteration for {df_list_names[i]}...")
    df = df_original.copy()  # Work with a copy to preserve the original DataFrame

    #status_types = ["all", "VRF"]  # Data status types ("all" for unfiltered data; "VRF" for verified data only)
    status_types = ["all"]  # Data status types ("all" for unfiltered data)

    # Configure parameters for each DataFrame based on index
    # Access the configuration using the index
    config = configurations.get(i)
    if config:
        response = config["response"]
        bin_width = config["bin_width"]
        fac_column = config["fac_column"]
        iqr_direction = config["iqr_direction"]
    
    # Additional configuration for df handling
    if i == 3:
        filter_limit = 10 if DATA_TYPE in ["energy", "water"] else None
        df = df[(df[response] >= 0) & (df[response1_column] / df[response0_column] < filter_limit)]
    elif i == 4:
        filter_limit = 10 if DATA_TYPE in ["energy", "water"] else None
        df = df[(df[response] >= 0) & (df[response1_output] / df[response0_output] < filter_limit)]
    elif i == 5:
        df = df[(df[response] <= 0)]
    else:
        pass

    # Retrieve unique facility types from the DataFrame
    facility_types = define_facility_types(df, fac_column)

    # Iterate over facility types and status types for detailed analysis
    for fac in facility_types:
        for stat in status_types:
            if VERBOSE: 
                print("-------------------------------------------------------------------------\n")
            print(f"Starting iteration {fac}.{stat}...")
            filtered_df = filter_df(df, fac_column, fac, stat)  # Apply filtering
            
            #compute sigmas
            print(response)
            sigma1, sigma2 = calc_sigma(filtered_df)

            print(sigma1, sigma2)
            
            
            # Plot histogram and mark IQR threshold if charts are enabled
#            if SHOW_CHARTS:
#                hist_plot(bin_counts_transf, bin_edges, bin_width, xmax=bins.max(), mark_vert=iqr_threshold)

            # Append the result summary
#            summary_row_dict = {
#                "dataframe": df_list_names[i],
#                "facility": fac,
#                "data_status": stat,
#                "IQR threshold type": iqr_direction,
#                "IQR threshold": iqr_threshold,
#                "IQR threshold (millions)": iqr_threshold / 1e6,
#                "total data points": number_total,
#                "flagged data points": number_flagged,
#                "percentage flagged": round(100 * number_flagged / number_total, 2),
#            }
#            summary_list.append(summary_row_dict)

    # Clean up DataFrame references and trigger garbage collection to free memory
    del df
    del df_original
    gc.collect()
    


********* Starting df iteration for df_all...
-------------------------------------------------------------------------

Starting iteration domestic.all...
response_mj
5642518.91455462 58342084.5273609
7071848.725681416 96783493.82551104
-------------------------------------------------------------------------

Starting iteration rawMaterialProcessing_Collection.all...
response_mj
256961125.18291578 1016115614.0701941
322053010.9322463 1685630879.6139708
-------------------------------------------------------------------------

Starting iteration materialProduction.all...
response_mj
216800286.4821479 518758194.10878706
271718864.02212644 860566277.0399501
-------------------------------------------------------------------------

Starting iteration hardComponentTrimProduction.all...
response_mj
29747125.71073323 79511810.08555129
37282493.19776335 131901882.54005913
-------------------------------------------------------------------------

Starting iteration vehicle.all...
response_mj

In [26]:
# Convert the list of dictionaries to a DataFrame
summary_df = pd.DataFrame(summary_list)

# Display the final DataFrame
if VERBOSE:
    print(f"Data type is {DATA_TYPE}.")
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        display(summary_df)


Data type is energy.


,dataframe,facility,data_status,IQR threshold type,IQR threshold,IQR threshold (millions),total data points,flagged data points,percentage flagged
0,df_all,rawMaterialProcessing_Collection,all,upper,6.600000e+08,660.000000,487,25,5.13
1,df_all,finalProductAssembly,all,upper,2.180000e+08,218.000000,7920,86,1.09
2,df_all,vehicle,all,upper,5.600000e+07,56.000000,7346,40,0.54
3,df_all,materialProduction,all,upper,1.059500e+09,1059.500000,2015,71,3.52
4,df_all,printingProductDyeingAndLaundering,all,upper,6.450000e+08,645.000000,2266,61,2.69
5,df_all,domestic,all,upper,6.450000e+07,64.500000,3744,28,0.75
6,df_all,hardComponentTrimProduction,all,upper,1.980000e+08,198.000000,641,18,2.81
7,df_sum,assessmentResponseSum,all,upper,1.035500e+09,1035.500000,11709,162,1.38
8,df_per,rawMaterialProcessing_Collection,all,upper,2.700000e+02,0.000270,483,13,2.69
9,df_per,finalProductAssembly,all,upper,8.100000e+02,0.000810,7910,40,0.51


In [27]:
# APPLY THRESHOLD COMPARISON AND ADD "flagged" COLUMN TO DATAFRAMES

for i, df in enumerate(df_list):
    if VERBOSE:
        print("---------------------------------------------------------------\n")

    # Fetch configuration parameters for the current DataFrame
    config = configurations.get(i)
    if config:
        response = config["response"]
        bin_width = config["bin_width"]
        fac_column = config["fac_column"]
        iqr_direction = config["iqr_direction"]

    print(f"********* Starting df iteration for {df_list_names[i]} ({iqr_direction})...")
        
    # Initialize flagged column to False unless it already exists, then do nothing to it.
    if "flagged" not in df.columns:
        df["flagged"] = False

    # Define initial parameters
    stat = "all"
    thresholds_dict = {}
    
    # Extract unique facility types for threshold assignment
    facility_types = define_facility_types(df, fac_column)

    if VERBOSE:
        display(facility_types)

    # Assign thresholds for each facility type based on the summary DataFrame
    for fac in facility_types:
        threshold = float(summary_df[(summary_df["dataframe"] == df_list_names[i]) & 
                                     (summary_df["facility"] == fac) & 
                                     (summary_df["data_status"] == "all") & 
                                     (summary_df["IQR threshold type"] == iqr_direction)]["IQR threshold"].iloc[0])
        thresholds_dict[fac] = threshold
        if VERBOSE:
            print(f"Starting iteration {df_list_names[i]}:{fac}.{stat}...")
            print(f"Current threshold: {threshold} ({threshold / 1e6}M)")

    if VERBOSE:
        display(thresholds_dict)
    
    # Apply the flagging condition using DataFrame apply()
    if iqr_direction.upper() == "UPPER":
        df["flagged"] = df.apply(
            lambda row: row["flagged"] or row[response] > thresholds_dict.get(row[fac_column], float("inf")),
            axis=1
        )
    elif iqr_direction.upper() == "LOWER":
        df["flagged"] = df.apply(
            lambda row: row["flagged"] or row[response] < thresholds_dict.get(row[fac_column], float("inf")),
            axis=1
        )

    # Display flagged rows for debugging if VERBOSE is enabled
    if VERBOSE:
        display(df)
        display(df[df["flagged"] == True])
        

---------------------------------------------------------------

********* Starting df iteration for df_all (upper)...


['rawMaterialProcessing_Collection',
 'finalProductAssembly',
 'vehicle',
 'materialProduction',
 'printingProductDyeingAndLaundering',
 'domestic',
 'hardComponentTrimProduction']

Starting iteration df_all:rawMaterialProcessing_Collection.all...
Current threshold: 660000000.0 (660.0M)
Starting iteration df_all:finalProductAssembly.all...
Current threshold: 218000000.0 (218.0M)
Starting iteration df_all:vehicle.all...
Current threshold: 56000000.0 (56.0M)
Starting iteration df_all:materialProduction.all...
Current threshold: 1059500000.0 (1059.5M)
Starting iteration df_all:printingProductDyeingAndLaundering.all...
Current threshold: 645000000.0 (645.0M)
Starting iteration df_all:domestic.all...
Current threshold: 64500000.0 (64.5M)
Starting iteration df_all:hardComponentTrimProduction.all...
Current threshold: 198000000.0 (198.0M)


{'rawMaterialProcessing_Collection': 660000000.0,
 'finalProductAssembly': 218000000.0,
 'vehicle': 56000000.0,
 'materialProduction': 1059500000.0,
 'printingProductDyeingAndLaundering': 645000000.0,
 'domestic': 64500000.0,
 'hardComponentTrimProduction': 198000000.0}

,assessment_id,account_id,rfi_pid,status,facility_posted,refid_mj,response_mj,refid_2,facility_type,facility_type_2,flagged
0,femsurvey:337325c0-0de2-4aa3-8d9e-79105e537e84,5a555c52d41b5d12cba674c7,fem2023,VRF,True,vehicle_total_mj,2469014.0,vehicle_total_mj,vehicle,vehicle,False
1,femsurvey:829a14d3-f506-4fc4-b76e-9a78b040db1a,51af1301aad8beff4fbd2a051f91f6f6,fem2023,VRF,True,vehicle_total_mj,2371698.0,vehicle_total_mj,vehicle,vehicle,False
2,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,bbf1677f4621395211a6d61f062cf6fa,fem2023,VRF,True,materialProduction_total_mj,229000000.0,materialProduction_total_mj,materialProduction,materialProduction,False
3,femsurvey:442b57b5-2678-412f-be7b-fb6798d4b1dd,5a1f5ee898d35f0486f6ebad,fem2023,VRF,True,domestic_total_mj,1587031.0,domestic_total_mj,domestic,domestic,False
4,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,60078743b8c28e000d9844cc,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,5491908.0,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering,False
...,...,...,...,...,...,...,...,...,...,...,...
24414,femsurvey:b2ce0262-573d-4812-ad4b-5cd615c097b0,5a77e834bec6594ba03155ed,fem2023,VRF,True,domestic_total_mj,5423090.0,domestic_total_mj,domestic,domestic,False
24415,femsurvey:470ec5af-4655-453d-9768-e1ede37b3370,5a1729ed0381837f8c762cde,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,206202.5,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering,False
24416,femsurvey:74f71a13-eec2-42fc-ad85-02f89bead82d,5a4f1aa6e77b90120dea3545,fem2023,VRF,True,printingProductDyeingAndLaundering_total_mj,61228218.0,printingProductDyeingAndLaundering_total_mj,printingProductDyeingAndLaundering,printingProductDyeingAndLaundering,False
24417,femsurvey:217c0b25-b7ea-49f4-bb67-cc7f78a93d58,606bd32ef4d279000b998654,fem2023,VRF,True,vehicle_total_mj,303839.5,vehicle_total_mj,vehicle,vehicle,False


...

['yoy_facility_placeholder']

Starting iteration df_yos:yoy_facility_placeholder.all...
Current threshold: -100.0 (-0.0001M)


{'yoy_facility_placeholder': -100.0}

,account_id,status,facility_placeholder,ResponseSum2023,ResponseSum2022,change,change%,flagged
0,0f95596cfb33b1171106a7a07900502c,VRF,yoy_facility_placeholder,2.088780e+07,17915367.20,2.972429e+06,16.591503,False
1,0f95596cfb33b1171106a7a079005076,VRF,yoy_facility_placeholder,4.032934e+07,10763363.90,2.956597e+07,274.690807,False
2,0f95596cfb33b1171106a7a079005c2b,VRF,yoy_facility_placeholder,1.771176e+07,20499249.10,-2.787487e+06,-13.597994,False
3,0f95596cfb33b1171106a7a07900cd0f,ASC,yoy_facility_placeholder,8.445816e+05,718470.00,1.261116e+05,17.552800,False
4,0f95596cfb33b1171106a7a07900d27d,ASC,yoy_facility_placeholder,1.014222e+06,1210535.26,-1.963131e+05,-16.217051,False
...,...,...,...,...,...,...,...,...
9105,bbf1677f4621395211a6d61f0653cad7,VRF,yoy_facility_placeholder,6.488776e+06,7771980.92,-1.283205e+06,-16.510658,False
9106,bbf1677f4621395211a6d61f06557463,VRF,yoy_facility_placeholder,2.518104e+04,94056.29,-6.887525e+04,-73.227693,False
9107,bbf1677f4621395211a6d61f065656c8,VRF,yoy_facility_placeholder,1.056024e+05,76680.00,2.892240e+04,37.718310,False
9108,bbf1677f4621395211a6d61f06569e5b,VRF,yoy_facility_placeholder,5.390616e+06,7062322.60,-1.671707e+06,-23.670778,False


,account_id,status,facility_placeholder,ResponseSum2023,ResponseSum2022,change,change%,flagged
89,0f95596cfb33b1171106a7a079438b45,VRF,yoy_facility_placeholder,5.063472e+05,6.331212e+03,5.000160e+05,7897.634576,True
102,0f95596cfb33b1171106a7a0794a9705,VRF,yoy_facility_placeholder,2.081124e+08,1.915901e+07,1.889534e+08,986.237725,True
239,0f95596cfb33b1171106a7a0798baf70,VRF,yoy_facility_placeholder,5.350255e+06,3.240000e+05,5.026255e+06,1551.313272,True
278,0f95596cfb33b1171106a7a07997d195,VRF,yoy_facility_placeholder,1.284656e+09,1.496746e+08,1.134981e+09,758.299134,True
349,0f95596cfb33b1171106a7a079bb3800,ASC,yoy_facility_placeholder,1.555200e+06,1.080000e+05,1.447200e+06,1340.000000,True
...,...,...,...,...,...,...,...,...
8956,6531e2bb6f3bf8000cd7465c,ASC,yoy_facility_placeholder,1.771031e+09,1.620858e+08,1.608945e+09,992.650203,True
8998,bbf1677f4621395211a6d61f0614c566,VRF,yoy_facility_placeholder,1.818995e+06,1.123583e+05,1.706636e+06,1518.923391,True
9003,bbf1677f4621395211a6d61f0617b0dc,VRF,yoy_facility_placeholder,7.378294e+06,6.267725e+05,6.751522e+06,1077.188632,True
9004,bbf1677f4621395211a6d61f0619b10e,VRF,yoy_facility_placeholder,2.362370e+06,1.751160e+04,2.344858e+06,13390.309580,True


In [28]:
# FLAGGED ASSESSMENTS STATISTICS -- prepare df_com

# Create combined DataFrame and add empty flagged column
df_com = df_all[["assessment_id", "account_id", "status", "flagged"]].copy()
df_com["flagged_all"] = df_com["flagged"]
df_com["flagged"] = ""  # Initialize a placeholder flagged column

if VERBOSE:
    print("*** df_com contains df_all only")
    display(df_com)

# Drop duplicate rows to remove redundancy
df_com = df_com.drop_duplicates()

if VERBOSE:
    print("*** Drop duplicate rows from df_com")
    display(df_com)

    # Check for duplicate assessment IDs and display them
    duplicate_rows = df_com[df_com["assessment_id"].duplicated(keep=False)]
    print("*** Remaining df_com duplicate assessment_id rows")
    display(duplicate_rows.sort_values("assessment_id"))

# Collapse multiple entries into one correctly flagged entry
# Group by specified columns and aggregate flagged_all column to get the maximum value (True if any True exists)
group_columns = ["assessment_id", "account_id", "status", "flagged"]
df_com = df_com.groupby(group_columns, as_index=False).agg({"flagged_all": "max"})

if VERBOSE:
    # Display the collapsed DataFrame
    print("*** df_com with duplicate assessment_id & account_id collapsed")
    display(df_com)
  

*** df_com contains df_all only


,assessment_id,account_id,status,flagged,flagged_all
0,femsurvey:337325c0-0de2-4aa3-8d9e-79105e537e84,5a555c52d41b5d12cba674c7,VRF,,False
1,femsurvey:829a14d3-f506-4fc4-b76e-9a78b040db1a,51af1301aad8beff4fbd2a051f91f6f6,VRF,,False
2,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,bbf1677f4621395211a6d61f062cf6fa,VRF,,False
3,femsurvey:442b57b5-2678-412f-be7b-fb6798d4b1dd,5a1f5ee898d35f0486f6ebad,VRF,,False
4,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,60078743b8c28e000d9844cc,VRF,,False
...,...,...,...,...,...
24414,femsurvey:b2ce0262-573d-4812-ad4b-5cd615c097b0,5a77e834bec6594ba03155ed,VRF,,False
24415,femsurvey:470ec5af-4655-453d-9768-e1ede37b3370,5a1729ed0381837f8c762cde,VRF,,False
24416,femsurvey:74f71a13-eec2-42fc-ad85-02f89bead82d,5a4f1aa6e77b90120dea3545,VRF,,False
24417,femsurvey:217c0b25-b7ea-49f4-bb67-cc7f78a93d58,606bd32ef4d279000b998654,VRF,,False


*** Drop duplicate rows from df_com


,assessment_id,account_id,status,flagged,flagged_all
0,femsurvey:337325c0-0de2-4aa3-8d9e-79105e537e84,5a555c52d41b5d12cba674c7,VRF,,False
1,femsurvey:829a14d3-f506-4fc4-b76e-9a78b040db1a,51af1301aad8beff4fbd2a051f91f6f6,VRF,,False
2,femsurvey:d8ec61cf-ca58-4520-a5d3-606292845c72,bbf1677f4621395211a6d61f062cf6fa,VRF,,False
3,femsurvey:442b57b5-2678-412f-be7b-fb6798d4b1dd,5a1f5ee898d35f0486f6ebad,VRF,,False
4,femsurvey:03771678-a1d4-47fd-8e71-437449abafcb,60078743b8c28e000d9844cc,VRF,,False
...,...,...,...,...,...
24373,femsurvey:6f8c91fb-cd9e-4408-b831-97eaa3185fe6,5ef1d0d3ba300d000a81bb02,VRF,,False
24398,femsurvey:e7f05eb5-26a2-4383-8252-68cd68ec8bbb,626a1eace1811d000b1aaf5e,ASC,,False
24401,femsurvey:f6fecc34-63c5-4be6-bb33-c4c9c2ff9e5a,5a0d826d5bbc963ed37c4cb8,VRF,,False
24405,femsurvey:4e518610-4238-4168-82cb-7a09fa951123,63db0be9e18e13000b971d07,VRF,,False


*** Remaining df_com duplicate assessment_id rows


,assessment_id,account_id,status,flagged,flagged_all
12961,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,False
2440,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True
4353,femsurvey:00fa9420-874b-4be4-bd0a-37ab8388ea65,5aebd4e79f9ac75adea8bb7d,VRF,,True
9540,femsurvey:00fa9420-874b-4be4-bd0a-37ab8388ea65,5aebd4e79f9ac75adea8bb7d,VRF,,False
22789,femsurvey:01a26896-fe66-4397-b288-7f635e52bdc3,6117689baf6c14000a5d467f,VRF,,True
...,...,...,...,...,...
8705,femsurvey:f99884b0-f1ae-45e4-aeeb-ed5fd213319a,51af1301aad8beff4fbd2a051f7adc5d,VRF,,True
18068,femsurvey:fbb5d9b7-82f3-45f8-8e4e-c8d773b7625f,6124885cdaee1f000aa98643,ASC,,False
6419,femsurvey:fbb5d9b7-82f3-45f8-8e4e-c8d773b7625f,6124885cdaee1f000aa98643,ASC,,True
1215,femsurvey:fcdc0503-22ca-42d2-8db2-5511047cff97,5ac88a021fc07f1b4d2f4065,VRF,,False


*** df_com with duplicate assessment_id & account_id collapsed


,assessment_id,account_id,status,flagged,flagged_all
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,,False
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,,False
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,,False
...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,,False


In [29]:
# FLAGGED ASSESSMENTS STATISTICS -- integrate df_sum

# df_sum should only have 1 entry per account_id
# Merge flagged information from df_sum into df_com
df_com = df_com.merge(
    df_sum[["assessment_id", "account_id", "flagged"]],  # Select only the relevant columns from df_sum
    on=["assessment_id", "account_id"],                  # Perform the merge based on these columns
    how="left",                              # Keep all rows from df_com (even if no match is found)
    suffixes=("", "_sum")                    # Append "_sum" suffix to distinguish conflicting column names
)

# Check the merge result for debugging if VERBOSE is enabled
if VERBOSE:
    print("*** Merged df_sum into df_com on account_id")
    display(df_com.sort_values("account_id"))

# Remove any duplicate rows that might have appeared after the merge
df_com = df_com.drop_duplicates()

# Provide detailed output after deduplication if VERBOSE is enabled
if VERBOSE:
    print("*** Dropped duplicates from df_com")
    display(df_com)


*** Merged df_sum into df_com on account_id


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum
10865,femsurvey:ece398ff-dda3-4e1a-8443-3de2d35e5df3,0f95596cfb33b1171106a7a07900502c,VRF,,False,False
11470,femsurvey:faa45f3f-ce9d-41b6-ab72-d7d94fb202b8,0f95596cfb33b1171106a7a079005076,VRF,,False,False
8436,femsurvey:b6f872ab-9c9f-4114-9c3e-9f9db7a1c560,0f95596cfb33b1171106a7a079005c2b,VRF,,False,False
9931,femsurvey:d83ab6bb-7401-41e2-95e1-87bade040626,0f95596cfb33b1171106a7a07900cd0f,ASC,,False,False
2115,femsurvey:2daaca59-a3e0-4eb0-a749-2a9b50f79a63,0f95596cfb33b1171106a7a07900d27d,ASC,,False,False
...,...,...,...,...,...,...
3024,femsurvey:41fee2b3-a70a-4c3b-a3a7-424ccab39193,bbf1677f4621395211a6d61f06557463,VRF,,False,False
10048,femsurvey:dafe9df5-0b40-4e6d-84e5-ea35ce4123e2,bbf1677f4621395211a6d61f06560077,ASC,,False,False
7509,femsurvey:a2a1838a-3aea-43c0-9bf4-c0087773b8ec,bbf1677f4621395211a6d61f065656c8,VRF,,False,False
4199,femsurvey:5b4a8c26-7684-4a64-ab9c-631bb968f83b,bbf1677f4621395211a6d61f06569e5b,VRF,,False,False


*** Dropped duplicates from df_com


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,,False,False
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,,False,False
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True,True
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,,False,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,,False,False
...,...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,,False,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,,False,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,,False,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,,False,False


In [30]:
# FLAGGED ASSESSMENTS STATISTICS -- integrate df_per

# Merge flagged information from df_per into df_com
df_com = df_com.merge(
    df_per[["assessment_id", "account_id", "flagged"]],  # Select only the relevant columns from df_per
    on=["assessment_id", "account_id"],                  # Perform the merge based on these columns
    how="left",                              # Keep all rows from df_com (even if no match is found)
    suffixes=("", "_per")                    # Append "_per" suffix to distinguish conflicting column names
)

# Debugging information for review after the merge
if VERBOSE:
    print("*** Merged df_per into df_com on account_id")
    display(df_com.sort_values("account_id"))

# Handle potential multiple entries per account_id from df_per by collapsing duplicated rows  
# Group by the essential columns and aggregate the flagged_per column to retain the maximum flag 
# (i.e., True if any entry is flagged True)
group_columns = ["assessment_id", "account_id", "status", "flagged", "flagged_all", "flagged_sum"]
df_com = df_com.groupby(group_columns, as_index=False).agg({"flagged_per": "max"})

# Display the grouped DataFrame to confirm duplicate entries are collapsed
if VERBOSE:
    print("*** df_com with duplicate assessment_id & account_id collapsed")
    display(df_com)

# Remove any remaining duplicate rows after aggregation
df_com = df_com.drop_duplicates()

# Confirm the results post deduplication
if VERBOSE:
    print("*** Dropped duplicates from df_com")
    display(df_com)
    

*** Merged df_per into df_com on account_id


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per
12152,femsurvey:ece398ff-dda3-4e1a-8443-3de2d35e5df3,0f95596cfb33b1171106a7a07900502c,VRF,,False,False,False
12151,femsurvey:ece398ff-dda3-4e1a-8443-3de2d35e5df3,0f95596cfb33b1171106a7a07900502c,VRF,,False,False,False
12827,femsurvey:faa45f3f-ce9d-41b6-ab72-d7d94fb202b8,0f95596cfb33b1171106a7a079005076,VRF,,False,False,False
12826,femsurvey:faa45f3f-ce9d-41b6-ab72-d7d94fb202b8,0f95596cfb33b1171106a7a079005076,VRF,,False,False,False
9447,femsurvey:b6f872ab-9c9f-4114-9c3e-9f9db7a1c560,0f95596cfb33b1171106a7a079005c2b,VRF,,False,False,False
...,...,...,...,...,...,...,...
3384,femsurvey:41fee2b3-a70a-4c3b-a3a7-424ccab39193,bbf1677f4621395211a6d61f06557463,VRF,,False,False,False
11237,femsurvey:dafe9df5-0b40-4e6d-84e5-ea35ce4123e2,bbf1677f4621395211a6d61f06560077,ASC,,False,False,False
8409,femsurvey:a2a1838a-3aea-43c0-9bf4-c0087773b8ec,bbf1677f4621395211a6d61f065656c8,VRF,,False,False,False
4689,femsurvey:5b4a8c26-7684-4a64-ab9c-631bb968f83b,bbf1677f4621395211a6d61f06569e5b,VRF,,False,False,False


*** df_com with duplicate assessment_id & account_id collapsed


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,,False,False,NaN
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,,False,False,False
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True,True,False
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,,False,False,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,,False,False,False
...,...,...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,,False,False,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,,False,False,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,,False,False,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,,False,False,True


*** Dropped duplicates from df_com


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,,False,False,NaN
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,,False,False,False
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True,True,False
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,,False,False,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,,False,False,False
...,...,...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,,False,False,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,,False,False,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,,False,False,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,,False,False,True


In [31]:
# FLAGGED ASSESSMENTS STATISTICS -- integrate df_yos

# df_yos should only have 1 entry per account_id
# Merge flagged information from df_yos into df_com
df_com = df_com.merge(
    df_yos[["account_id", "flagged"]],  # Select only relevant columns from df_yos
    on=["account_id"],                  # Merge on account_id since assessment_id is not relevant here
    how="left",                 # Keep all rows from df_com, even if no matching entry in df_yos
    suffixes=("", "_yos")       # Add "_yos" suffix to distinguish conflicting `flagged` column from df_yos
)

if VERBOSE:
    print("*** Merged df_yos into df_com on account_id")
    display(df_com.sort_values("account_id"))

# Remove duplicate rows after the merge
df_com = df_com.drop_duplicates()

# Confirm the state of the DataFrame post deduplication
if VERBOSE:
    print("*** Dropped duplicates from df_com")
    display(df_com)
    

*** Merged df_yos into df_com on account_id


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos
10865,femsurvey:ece398ff-dda3-4e1a-8443-3de2d35e5df3,0f95596cfb33b1171106a7a07900502c,VRF,,False,False,False,False
11470,femsurvey:faa45f3f-ce9d-41b6-ab72-d7d94fb202b8,0f95596cfb33b1171106a7a079005076,VRF,,False,False,False,False
8436,femsurvey:b6f872ab-9c9f-4114-9c3e-9f9db7a1c560,0f95596cfb33b1171106a7a079005c2b,VRF,,False,False,False,False
9931,femsurvey:d83ab6bb-7401-41e2-95e1-87bade040626,0f95596cfb33b1171106a7a07900cd0f,ASC,,False,False,False,False
2115,femsurvey:2daaca59-a3e0-4eb0-a749-2a9b50f79a63,0f95596cfb33b1171106a7a07900d27d,ASC,,False,False,False,False
...,...,...,...,...,...,...,...,...
3024,femsurvey:41fee2b3-a70a-4c3b-a3a7-424ccab39193,bbf1677f4621395211a6d61f06557463,VRF,,False,False,False,False
10048,femsurvey:dafe9df5-0b40-4e6d-84e5-ea35ce4123e2,bbf1677f4621395211a6d61f06560077,ASC,,False,False,False,NaN
7509,femsurvey:a2a1838a-3aea-43c0-9bf4-c0087773b8ec,bbf1677f4621395211a6d61f065656c8,VRF,,False,False,False,False
4199,femsurvey:5b4a8c26-7684-4a64-ab9c-631bb968f83b,bbf1677f4621395211a6d61f06569e5b,VRF,,False,False,False,False


*** Dropped duplicates from df_com


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,,False,False,NaN,False
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,,False,False,False,NaN
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,,True,True,False,False
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,,False,False,False,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,,False,False,False,False
...,...,...,...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,,False,False,False,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,,False,False,False,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,,False,False,False,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,,False,False,True,NaN


In [32]:
# FLAGGED ASSESSMENTS STATISTICS -- Final Processing of df_com

def flag_outliers(df, account_id_colname="", assessment_id_colname="", VERBOSE=True):

    # Check for duplicate entries in account_id and assessment_id for debugging purposes
    if VERBOSE:
        # Identify rows with duplicate account_id values and display them for review
        if account_id_colname:
            duplicate_rows = df[df[account_id_colname].duplicated(keep=False)]
            print(f"*** Duplicate rows based on {account_id_colname} (should be an empty dataframe)")
            display(duplicate_rows.sort_values(account_id_colname))

        # Identify rows with duplicate assessment_id values and display them for review
        if assessment_id_colname:
            duplicate_rows = df[df[assessment_id_colname].duplicated(keep=False)]
            print(f"\n*** Duplicate rows based on {assessment_id_colname} (should be an empty dataframe)")
            display(duplicate_rows.sort_values(assessment_id_colname))

    # List of the boolean flag columns representing potential flagged assessments
    flag_columns = ["flagged_all", "flagged_sum", "flagged_per", "flagged_yos"]

    # Ensure NaN values are treated as False, then check if any flag columns are True
    # Update the 'flagged' column to indicate if any flags were raised
    df["flagged"] = (df[flag_columns].notna() & 
                    df[flag_columns].astype(bool)).any(axis=1)
    # This more compact version of the previous 2 lines is deprecated because of
    # FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. 
    #df['flagged'] = df[flag_columns].fillna(False).any(axis=1) 

    # Explicit conversion to boolean type to standardize the column values (ensures no NaN remains)
    df["flagged"] = df["flagged"].astype(bool)

    return df


In [33]:
df_com = flag_outliers(df_com, "account_id", "assessment_id", VERBOSE)

if VERBOSE:
    # Display the DataFrame after processing the flagged column
    print("\n*** df_com with flagged column filled")
    display(df_com)

    # Check for rows where 'flagged_yos' is True to confirm successful merge and flag application
    print("*** Test for flagged_yos = True")
    display(df_com[df_com["flagged_yos"] == True])
    

*** Duplicate rows based on account_id (should be an empty dataframe)


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos



*** Duplicate rows based on assessment_id (should be an empty dataframe)


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos



*** df_com with flagged column filled


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos
0,femsurvey:000f6133-78d5-4a0e-ba7d-522d6da6c832,5a6e9b5b963a52483a65a198,VRF,False,False,False,NaN,False
1,femsurvey:000f687e-a6b1-48de-aa5a-a44d5fdbf098,66b097aedf7d9e000cf2eb7e,ASC,False,False,False,False,NaN
2,femsurvey:000f8eff-a94c-467c-a02b-98a1aee7eb8d,5a77fba4d02d57577cfc1f11,VRF,True,True,True,False,False
3,femsurvey:00131c99-c664-4bc2-a365-9832ff70a9c6,51af1301aad8beff4fbd2a051f7e0c8c,VRF,False,False,False,False,False
4,femsurvey:001e124f-3981-4a39-9747-7db7d212e864,6038b07cc14f64000b610838,VRF,False,False,False,False,False
...,...,...,...,...,...,...,...,...
11704,femsurvey:ffea25b7-5e06-4159-acdb-af2d17cd8c21,5a16e4a7e836fc783cfb34fe,VRF,False,False,False,False,False
11705,femsurvey:fff0826c-28af-4687-9c7a-4dfe18cd55ca,5ae34e0fba2da67747ec4d33,VRF,False,False,False,False,False
11706,femsurvey:fff5effe-e53b-4fab-8c81-d55fbc24615a,62259ecf31d323000b100fb2,VRF,False,False,False,False,False
11707,femsurvey:fffb564d-aacc-401a-ab6a-e1b4189e2f94,660c0c0009ab8c000cb10f84,VRF,True,False,False,True,NaN


*** Test for flagged_yos = True


,assessment_id,account_id,status,flagged,flagged_all,flagged_sum,flagged_per,flagged_yos
53,femsurvey:01245ae8-8f88-4de6-a1ac-c06f3a3b9152,5a55697ade45ba1939ea2cf7,VRF,True,False,False,False,True
133,femsurvey:030a113a-f4d5-45a0-9b8a-dbf6a0201586,62442acfc50da4000bd4a023,VRF,True,False,False,False,True
137,femsurvey:0322eb90-8f7b-4a07-8905-b63dff7af741,60af482bc854c6000a78f95c,VRF,True,False,False,False,True
138,femsurvey:03250088-5b54-4e2b-85ae-b12f334526ee,5ed74327bea5ed00091d710f,VRF,True,False,False,False,True
285,femsurvey:064d1a15-116b-45cf-87c2-b271d451760b,bbf1677f4621395211a6d61f0617b0dc,VRF,True,False,False,False,True
...,...,...,...,...,...,...,...,...
11521,femsurvey:fbb5d9b7-82f3-45f8-8e4e-c8d773b7625f,6124885cdaee1f000aa98643,ASC,True,True,True,False,True
11546,femsurvey:fc4db854-31dd-42bc-8667-7abe08b1c196,649113739949bb000c1f8a33,ASC,True,False,False,False,True
11572,femsurvey:fcf47a1a-6f8b-4109-9ffa-6311b9fec524,600945f25fa566000bb5cef4,ASC,True,False,False,False,True
11579,femsurvey:fd2153f9-3871-4de3-ba17-0e70a3071002,5b021457d0ce772e1a74d086,VRF,True,False,False,False,True


In [34]:
# FLAGGED ASSESSMENTS STATISTICS

print("Note: the available pool of assessments has already been cleaned; i.e., by removing those assessments outside the Cascale realistic ranges.\n")

print(f"Data type is {DATA_TYPE}.\n")

# **Overall Flagged Assessments Statistics**

# Count the total number of unique assessments
count_unique_assessments = len(df_com)

# Count the number of flagged unique assessments
count_unique_flagged = df_com[df_com["flagged"] == True].shape[0]

# Display the flagged assessment statistics
print(f"Unique assessments flagged by facility type OR facility sum OR per unit OR yoy sum on facility type:\n"
      f"{count_unique_flagged} ({round(100 * count_unique_flagged / count_unique_assessments, 2)}% of {count_unique_assessments} unique assessments)\n")

# **VRF-Specific Flagged Assessments Statistics**

# Count the number of VRF assessments
count_unique_assessments_VRF = len(df_com[df_com["status"] == "VRF"])

# Count the number of flagged VRF assessments
count_unique_flagged_VRF = df_com[(df_com["flagged"] == True) & (df_com["status"] == "VRF")].shape[0]

# Display VRF flagged assessment statistics
print(f"Unique VRF assessments flagged by facility type OR facility sum OR per unit OR yoy sum on facility type:\n"
      f"{count_unique_flagged_VRF} ({round(100 * count_unique_flagged_VRF / count_unique_assessments_VRF, 2)}% of {count_unique_assessments_VRF} unique VRF assessments)")


Note: the available pool of assessments has already been cleaned; i.e., by removing those assessments outside the Cascale realistic ranges.

Data type is energy.

Unique assessments flagged by facility type OR facility sum OR per unit OR yoy sum on facility type:
670 (5.72% of 11709 unique assessments)

Unique VRF assessments flagged by facility type OR facility sum OR per unit OR yoy sum on facility type:
454 (4.96% of 9145 unique VRF assessments)


In [35]:
# INDIVIDUAL TEST FLAGGING STATISTICS

print(f"Data type is {DATA_TYPE}.\n")

# List of flag types and their corresponding descriptions
flag_types = {
    "flagged_all": "facility type",
    "flagged_sum": "facility sum",
    "flagged_per": "per unit",
    "flagged_yos": "yoy sum on facility type",
}

# Total count of unique assessments (only calculated once)
count_unique_assessments = len(df_com)

# Iterate through each flag type and print the corresponding statistics
for flag_column, description in flag_types.items():
    count_unique_flagged = df_com[df_com[flag_column] == True].shape[0]
    percentage_flagged = round(100 * count_unique_flagged / count_unique_assessments, 2)
    print(f"Unique assessments flagged by {description}:\n"
          f"{count_unique_flagged} ({percentage_flagged}% of {count_unique_assessments} unique assessments)\n")


Data type is energy.

Unique assessments flagged by facility type:
296 (2.53% of 11709 unique assessments)

Unique assessments flagged by facility sum:
163 (1.39% of 11709 unique assessments)

Unique assessments flagged by per unit:
131 (1.12% of 11709 unique assessments)

Unique assessments flagged by yoy sum on facility type:
286 (2.44% of 11709 unique assessments)

